# Lab 3 - Computaional Intelligence 2024

In [119]:
from collections import namedtuple, deque
from queue import *
from random import *
from tqdm.auto import tqdm
import numpy as np

In [120]:
PUZZLE_DIM = 2
RANDOMIZE_STEPS = 100_000

action = namedtuple('Action', ('pos1', 'pos2'))

# Define a Node for gougeous visual plot of solution path

In [121]:
#   Default game: search for number ordered increasing
default_game = True 

#   If we wanna search a Specific configuration, ENTER here the configuration following the rules:
#   -   MUST contain ONLY one "0";
#   -   write number from 1 to PUZZLE_DIM**2 - 1 in a list

GOAL_STATE = [1, 2, 3, 0]

if not default_game:
    if GOAL_STATE.count(0) != 1:
        print("You insered too many 0s in your goal state!\n", GOAL_STATE)
        exit(1)

    if len(set(GOAL_STATE)) != PUZZLE_DIM**2:
        print("You haven't created a feasible solution!\n", GOAL_STATE)
        exit(1)

    GOAL_STATE = np.array(GOAL_STATE).reshape((PUZZLE_DIM, PUZZLE_DIM))
else:
    GOAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

GOAL_STATE

array([[1, 2],
       [3, 0]])

In [122]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [123]:
#   Initial state of the game   #
INITIAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for _ in range(RANDOMIZE_STEPS):
    INITIAL_STATE = do_action(INITIAL_STATE, choice(available_actions(INITIAL_STATE)))
print(INITIAL_STATE)

[[0 2]
 [1 3]]


### Utility functions

In [124]:
def solution_check(state: np.ndarray) -> bool:
    return np.array_equal(state, GOAL_STATE)

In [125]:
def avoid_loop(state: np.ndarray, visited_state: list[np.ndarray]) -> bool:
    return any(np.array_equal(state, s) for s in visited_state)

### Randomized Solution

In [107]:
state = INITIAL_STATE

for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    #print(state)
    state = do_action(state, choice(available_actions(state)))
    if solution_check(state):
        break
print(f"Solved in {r:,} steps")

Randomizing:   0%|          | 11/100000 [00:00<?, ?it/s]

Solved in 11 steps


## PATH SEARCH STRATEGY

Mi serve tener conto di:
- Costo --> Quanti nodi ho esplorato -> nodi = azioni fatte per cambiare lo stato del gioco
- Qualità --> Minor numero di nodi necessari per arrivare alla soluzione -> nodi = azioni

OBIETTIVO --> ___Qualità vs Costo___

### Depth-First Strategy

Esploro in profondità ciascun nodo fino alla foglia, poi passo al figlio successivo.<br>
Criticità -> Soluzioni :
- ...

In [118]:
frontier = deque([INITIAL_STATE])

explored_state = list()
#   action_sequence = list()
state = frontier.pop()
explored_state.append(state)
explored_node = 1

with tqdm(total=None, desc="Depth-First") as pbar:
    while not solution_check(state):
        print(state, "\n")
        for a in available_actions(state):
            new_state = do_action(state, a)
            if not avoid_loop(new_state, explored_state):
                frontier.appendleft(new_state)

        print(len(frontier))
        state = frontier.popleft()
        explored_state.append(state)
        explored_node += 1
        pbar.update(1)

print(f"Solved in {explored_node:,} steps")

Depth-First: 2it [00:00, ?it/s]

[[0 1]
 [3 2]] 

2
[[1 0]
 [3 2]] 

2
Solved in 3 steps


### Breadth-First Strategy

Esploro prima tutti i figli di nodo allo stesso livello, poi passo ai figli, fino alle foglie

In [94]:
frontier = deque([INITIAL_STATE])

explored_state = list()
#   action_sequence = list()
state = frontier.pop()
explored_state.append(state)
explored_node = 1

with tqdm(total=None, desc="Depth-First") as pbar:
    
    while not solution_check(state):
        actions = available_actions(state)

        for a in actions:
            new_state = do_action(state, a)
            if not avoid_loop(new_state, explored_state):
                frontier.append(new_state)

        #print(len(frontier))
        state = frontier.popleft()
        explored_state.append(state)
        explored_node += 1
        pbar.update(1)

print(f"Solved in {explored_node:,} steps")

Depth-First: 7it [00:00, ?it/s]

Solved in 8 steps


### A* Strategy

In [128]:
def heuristic(state: np.ndarray) -> int:
    '''
        Use the classic distance definition, of each point from its correct position, as cost
    '''
    n = state.shape[0]
    distance = 0

    for (i, j), tile in np.ndenumerate(state):
        if tile == 0:
            continue

        target_i, target_j = divmod(tile-1, n)
        distance += min(abs(i - target_i) + abs(j - target_j))

    return distance

In [129]:
class Node:
    def __init__(self, board: np.ndarray, previous_state: "Node") -> None:
        self.board = board      # Game state
        self.previous_state = previous_state    # Parent Node
        if previous_state is None:      # Actual Cost to reach this Node
            self.g = 0
        else:
            self.g = previous_state.g
        self.f = (self.g + 1) + (heuristic(board))    # f(x) = g(x) + h(x)

    def __eq__(self, other):
        return (self.g, self.f) == (other.g, other.f)

    def __lt__(self, other):
        return (self.g, self.f) < (other.g, other.f)

    def __repr__(self):
        """
        Rappresentazione leggibile del nodo.
        """
        stato_str = '\n'.join([' '.join(map(str, riga)) for riga in self.stato])
        return f"Stato:\n{stato_str}\nEuristica: {self.euristica}\n"

In [130]:
def better_path_search(frontier: list[(np.ndarray, int)], state: tuple[np.ndarray, int] ) -> tuple[np.ndarray, int] :
    return None, 0

def better_path(state: tuple[np.ndarray, int], explored_state: list[(np.ndarray, int)]) -> list[(np.ndarray, int)]:
    return None

In [133]:
frontier = PriorityQueue()

explored_state = list()
state = Node(INITIAL_STATE, None)
explored_state.append(state)

with tqdm(total=None) as pbar:

    while not solution_check(state.board):
        for a in available_actions(state[0]):
            new_state = do_action(state.board, a)
            frontier.put(Node(new_state, state))

        state = frontier.get()
        explored_state.append(state)
        explored_node += 1
        pbar.update(1)


print(f"Solved in {explored_node:,} steps")

TypeError: 'numpy.int64' object is not iterable

# Show Solution

In [45]:
print(state,"\n", GOAL_STATE)

[[1 2]
 [3 0]] 
 [[1 2]
 [3 0]]
